In [0]:
import pandas as pd
import numpy as np
import sklearn
import sklearn.metrics
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os, shutil
import pathlib
from google.colab import files

nn_root = pathlib.Path('/content/drive/My Drive/neural_networks') # insert your drive directory here...

In [0]:
%tensorflow_version 2.x
import tensorflow as tf
print(tf.__version__)
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model
from tensorflow.keras import regularizers
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt


2.2.0-rc3


In [0]:
path = "/content/drive/My Drive/data/TriageVitalSignAnalysisDeIdentified.csv" # insert your drive directory here...
df = pd.read_csv(path, low_memory=False)
# Dataset is now stored in a Pandas Dataframe

In [0]:
df

,Age,Sex,CC,Tr Temp,Tr HR,Tr SBP,Tr DBP,Tr RR,Tr O2,ESI,Mortality,floor2icu,ICU,Trigger,Trigger_Vital,Dispo,ICD9 CODES,ICD10 CODES
0,90,F,"s/p Fall, Pelvic pain",98.2,75.0,139.0,84.0,16.0,93.0,2,0,0,0,0,0,ADMITTED,808.2 E888.9 599.0,NaN
1,48,F,Vision changes,98.0,124.0,190.0,99.0,18.0,100.0,2,0,0,0,0,0,HOME,379.43,NaN
2,45,M,Testicular pain,97.6,103.0,112.0,77.0,14.0,96.0,3,0,0,0,0,0,ELOPED,608.9,NaN
3,42,F,"SI, HI",98.5,85.0,147.0,81.0,16.0,100.0,2,0,0,0,0,0,TX/TRANSFER TO ANOTHER FACILITY,311 V62.84,NaN
4,47,M,LEG CRAMPS,96.7,87.0,154.0,90.0,18.0,97.0,4,0,0,0,0,0,LEFT WITHOUT BEING SEEN,729.5,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445920,64,M,"Fever, N/V",98.2,103.0,113.0,64.0,20.0,94.0,2,0,0,0,0,0,ADMITTED,NaN,D64.9
445921,31,F,"Abd pain, Transfer",97.3,82.0,122.0,86.0,16.0,100.0,2,0,0,0,0,0,HOME,NaN,
445922,31,F,Altered mental status,98.4,78.0,115.0,65.0,18.0,100.0,2,0,0,0,0,0,ADMITTED,NaN,R26.2
445923,39,M,"n/v/d, Palpitations",98.6,110.0,143.0,78.0,18.0,100.0,3,0,0,0,0,0,HOME,NaN,


In [0]:
# change CC words to integers
tokenizer = tfds.features.text.Tokenizer()

vocabulary_set = set()
for CC in df['CC'].dropna():
  some_tokens = tokenizer.tokenize(CC)
  vocabulary_set.update(some_tokens)

vocab_size = len(vocabulary_set)
vocab_size

5249

In [0]:
# create encoder
encoder = tfds.features.text.TokenTextEncoder(vocabulary_set)
#encoded_example = encoder.encode(df['CC'][40])
#print(encoded_example)


In [0]:
def tokenize_language(text):
  return encoder.encode(text)

In [0]:
df['CC_t'] = df['CC'].apply(tokenize_language) # convert Chief Complaint to embedded word vector

In [0]:
# get max length of triage CC, then zero pad the endings
maximum_cc_len = df['CC_t'].str.len().max()
df['CC_tp'] = tf.keras.preprocessing.sequence.pad_sequences(df['CC_t'], maxlen=maximum_cc_len, padding='post', value=0).tolist()

16


In [0]:
df

,Age,Sex,CC,Tr Temp,Tr HR,Tr SBP,Tr DBP,Tr RR,Tr O2,ESI,Mortality,floor2icu,ICU,Trigger,Trigger_Vital,Dispo,ICD9 CODES,ICD10 CODES,Positive,CC_t,CC_tp
0,1.837880,1.0,"s/p fall, pelvic pain",0.250746,-0.274883,0.286288,0.464963,-0.159303,-0.024808,2,0,0,0,0.0,0,ADMITTED,808.2 E888.9 599.0,NaN,0.0,"[2509, 373, 4614, 4416, 5199]","[2509, 373, 4614, 4416, 5199, 0, 0, 0, 0, 0, 0..."
1,-0.201111,1.0,vision changes,0.241958,1.751981,1.730294,1.150813,0.295105,0.302073,2,0,0,0,0.0,0,HOME,379.43,NaN,0.0,"[1963, 3212]","[1963, 3212, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,-0.346753,0.0,testicular pain,0.224381,0.883325,-0.478186,0.144899,-0.613710,0.115284,3,0,0,0,0.0,0,ELOPED,608.9,NaN,0.0,"[1885, 5199]","[1885, 5199, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,-0.492395,1.0,"si, hi",0.263929,0.138763,0.512799,0.327793,-0.159303,0.302073,2,0,0,0,0.0,0,TX/TRANSFER TO ANOTHER FACILITY,311 V62.84,NaN,0.0,"[1026, 835]","[1026, 835, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
4,-0.249658,0.0,leg cramps,0.184833,0.221492,0.710996,0.739303,0.295105,0.161981,4,0,0,0,0.0,0,LEFT WITHOUT BEING SEEN,729.5,NaN,0.0,"[393, 3539]","[393, 3539, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445920,0.575648,0.0,"fever, n/v",0.250746,0.883325,-0.449872,-0.449505,0.749512,0.021889,2,0,0,0,0.0,0,ADMITTED,NaN,D64.9,0.0,"[4123, 1123, 1730]","[4123, 1123, 1730, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
445921,-1.026417,1.0,"abd pain, transfer",0.211198,0.014669,-0.195047,0.556409,-0.159303,0.302073,2,0,0,0,0.0,0,HOME,NaN,,0.0,"[1451, 5199, 2938]","[1451, 5199, 2938, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
445922,-1.026417,1.0,altered mental status,0.259534,-0.150789,-0.393244,-0.403781,0.295105,0.302073,2,0,0,0,0.0,0,ADMITTED,NaN,R26.2,0.0,"[5150, 1734, 4910]","[5150, 1734, 4910, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
445923,-0.638037,0.0,"n/v/d, palpitations",0.268323,1.172877,0.399543,0.190623,0.295105,0.302073,3,0,0,0,0.0,0,HOME,NaN,,0.0,"[1123, 1730, 3471, 3093]","[1123, 1730, 3471, 3093, 0, 0, 0, 0, 0, 0, 0, ..."


In [0]:
x_data = df[['Age', 'Sex',	'Tr Temp',	'Tr HR',	'Tr SBP',	'Tr DBP',	'Tr RR',	'Tr O2', 'CC_tp', 'Trigger', 'Trigger_Vital', 'ESI', 'Positive']]
x_data.dropna()


x_pos = x_data[x_data['Positive'] == 1]
x_neg = x_data[x_data['Positive'] == 0]

x_total = x_data


In [0]:
# main neural network model
# inputs for vital signs (8 items), inputs for word vector (16 items)
inputA = tf.keras.Input(shape=(8,))
inputB = tf.keras.Input(shape=(16,))

# branch A - vital signs (normalized inputs)
x = tf.keras.layers.Dense(8, activation="relu", kernel_regularizer=regularizers.l2(0.01))(inputA)
x = tf.keras.layers.Dense(8, activation="relu", kernel_regularizer=regularizers.l2(0.01))(x)
x = Model(inputs=inputA, outputs=x)

# branch B - chief complaint text
y = tf.keras.layers.Embedding(vocab_size+1, 16)(inputB)
y = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32))(y)


y = tf.keras.layers.Dense(32, activation="relu", kernel_regularizer=regularizers.l2(0.01))(y)
y = Model(inputs=inputB, outputs=y)

# combine vital sign data and freetext
combined = tf.keras.layers.concatenate([x.output, y.output])

# apply a FC layer and then a regression prediction on the
# combined outputs
z = tf.keras.layers.Dense(8, activation="relu", kernel_regularizer=regularizers.l2(0.01))(combined)
z = tf.keras.layers.Dense(1, activation="sigmoid")(z)

# predict the primary outcome!
nn_model = Model(inputs=[x.input, y.input], outputs=z)

nn_model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy', tf.keras.metrics.AUC(name='auc')])

In [0]:
# logistic regression
# inputs for vital signs (8 items)
inputL = tf.keras.Input(shape=(8,))
z_logit = tf.keras.layers.Dense(1, activation="sigmoid")(inputL)

# and tell us whether we should be worried or not!
model_logit = Model(inputs=inputL, outputs=z_logit)

model_logit.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy', tf.keras.metrics.AUC(name='auc')])

In [0]:
dataset = x_total.sample(frac=1,random_state=8) # shuffle
# Please note that for a cross-validation, you will need to re-run this in a loop and set an appropriate fraction for
# the number of folds of cross-validation!

full_vitals = dataset[['Age', 'Sex',	'Tr Temp',	'Tr HR',	'Tr SBP',	'Tr DBP',	'Tr RR',	'Tr O2']]
full_text = pd.DataFrame(dataset['CC_tp'].values.tolist())
all_true_labels = dataset['Positive'].values

mask1 = np.random.rand(len(dataset)) < 0.8
train_dataset = dataset[mask1]

## Don't forget to balance the training set!
training_fraction = len(train_dataset[train_dataset['Positive'] == 1]) / len(train_dataset)
maskpos = (np.random.rand(len(train_dataset)) < training_fraction) + (train_dataset['Positive'] == 1).values
train_dataset = train_dataset[maskpos]

remainder = dataset[~mask1]
mask2 = np.random.rand(len(remainder)) < 0.5
test_dataset = remainder[mask2]
val_dataset = remainder[~mask2]

# Manual separation of data and label -  can be done more artfully with scikit-learn 
train_CC_dataset = pd.DataFrame(train_dataset.pop('CC_tp').values.tolist())
val_CC_dataset = pd.DataFrame(val_dataset.pop('CC_tp').values.tolist())
test_CC_dataset = pd.DataFrame(test_dataset.pop('CC_tp').values.tolist())

train_labels = train_dataset.pop('Positive')
val_labels = val_dataset.pop('Positive')
test_labels = test_dataset.pop('Positive')

train_trigger_labels = train_dataset.pop('Trigger')
val_trigger_labels = val_dataset.pop('Trigger')
test_trigger_labels = test_dataset.pop('Trigger')

train_trigger_vital_labels = train_dataset.pop('Trigger_Vital')
val_trigger_vital_labels = val_dataset.pop('Trigger_Vital')
test_trigger_vital_labels = test_dataset.pop('Trigger_Vital')

train_ESI_labels = train_dataset.pop('ESI')
val_ESI_labels = val_dataset.pop('ESI')
test_ESI_labels = test_dataset.pop('ESI')

In [0]:
nn_model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy', tf.keras.metrics.AUC(name='auc')])

nn_history = nn_model.fit([train_dataset, train_CC_dataset.values], train_labels, epochs=5, validation_data=([val_dataset, val_CC_dataset.values], val_labels))


Train on 90830 samples, validate on 44613 samples
Epoch 1/5
90830/90830 [==============================] - 43s 470us/sample - loss: 0.5760 - accuracy: 0.7561 - auc: 0.8282 - val_loss: 0.5746 - val_accuracy: 0.7214 - val_auc: 0.8406
Epoch 2/5
90830/90830 [==============================] - 38s 417us/sample - loss: 0.5124 - accuracy: 0.7686 - auc: 0.8417 - val_loss: 0.6118 - val_accuracy: 0.6885 - val_auc: 0.8412
Epoch 3/5
90830/90830 [==============================] - 38s 418us/sample - loss: 0.5046 - accuracy: 0.7704 - auc: 0.8443 - val_loss: 0.5971 - val_accuracy: 0.7021 - val_auc: 0.8419
Epoch 4/5
90830/90830 [==============================] - 38s 418us/sample - loss: 0.5000 - accuracy: 0.7720 - auc: 0.8459 - val_loss: 0.4929 - val_accuracy: 0.7589 - val_auc: 0.8415
Epoch 5/5
90830/90830 [==============================] - 38s 420us/sample - loss: 0.4970 - accuracy: 0.7729 - auc: 0.8470 - val_loss: 0.5187 - val_accuracy: 0.7369 - val_auc: 0.8424


In [0]:
### optional - save nn_full model
nn_model.save('neural_network_full_model.h5')
files.download('neural_network_full_model.h5')

In [0]:
nn_guesses = nn_model.predict([test_dataset, test_CC_dataset.values])
nn_guesses_full = nn_model.predict([full_vitals, full_text.values])

In [0]:
vitals_nn_model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(8, activation='relu', input_shape=(8,), kernel_regularizer=regularizers.l2(0.01)),
  tf.keras.layers.Dense(8, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

vitals_nn_model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy', tf.keras.metrics.AUC(name='auc')])

vitals_nn_history = vitals_nn_model.fit(train_dataset, train_labels, epochs=5, validation_data=(test_dataset, test_labels))

Train on 90830 samples, validate on 44332 samples
Epoch 1/5
90830/90830 [==============================] - 8s 91us/sample - loss: 0.5833 - accuracy: 0.7220 - auc: 0.7950 - val_loss: 0.5755 - val_accuracy: 0.6976 - val_auc: 0.8152
Epoch 2/5
90830/90830 [==============================] - 7s 80us/sample - loss: 0.5428 - accuracy: 0.7377 - auc: 0.8104 - val_loss: 0.6168 - val_accuracy: 0.6606 - val_auc: 0.8161
Epoch 3/5
90830/90830 [==============================] - 7s 79us/sample - loss: 0.5394 - accuracy: 0.7381 - auc: 0.8110 - val_loss: 0.5953 - val_accuracy: 0.6700 - val_auc: 0.8173
Epoch 4/5
90830/90830 [==============================] - 7s 79us/sample - loss: 0.5372 - accuracy: 0.7381 - auc: 0.8120 - val_loss: 0.5559 - val_accuracy: 0.6996 - val_auc: 0.8174
Epoch 5/5
90830/90830 [==============================] - 7s 79us/sample - loss: 0.5358 - accuracy: 0.7388 - auc: 0.8127 - val_loss: 0.5628 - val_accuracy: 0.6873 - val_auc: 0.8183


In [0]:
### optional - save nn model
vitals_nn_model.save('neural_network_vitals_model.h5')
files.download('neural_network_vitals_model.h5')

In [0]:
vital_nn_guesses = vitals_nn_model.predict(test_dataset)
vital_nn_guesses_full = vitals_nn_model.predict(full_vitals)